In [15]:
from az_fields_dicts import *

In [5]:
import pandas as pd
import numpy as np
import geopandas as gp
import os
import maup
from op_verification.reference_data import *
pd.set_option("display.max_columns", None)

maup.progress.enabled = True

def import_block_file(block_cols, block_file_path):
    print('File imports initiated')
    block_gdf = gp.read_file(block_file_path)[block_cols]
    print('block shape: ', block_gdf.shape)
    return block_gdf


def import_prec_files(prec_gen_file_path, prec_prim_file_path):
    precinct_gen_gdf = gp.read_file(prec_gen_file_path)
    precinct_prim_gdf = gp.read_file(prec_prim_file_path)
    precinct_gdf = precinct_gen_gdf.merge(precinct_prim_gdf, how='outer', on=['UNIQUE_ID', 'geometry'], indicator=True)
    print('precinct shape post merge: ', precinct_gdf.shape)
    return precinct_gdf

In [6]:
cd ../PycharmProjects/AzDisag20/

/Users/lilyfalk/PycharmProjects/AzDisag20


### Load and allocate votes

In [8]:
#Import files
block_cols = ['STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'BLOCKCE20', "CD116","SLDU18","SLDL18", 'GEOID20', 'P0040001', 'P0050003', 'NAME20', 'ALAND20',
              'AWATER20', 'geometry']
block_file_path = '/Users/lilyfalk/election_disag/pl2020/az_pl2020_b/az_pl2020_b.shp'
prec_gen_file_path = './az_gen_20_prec/az_gen_20_prec.shp'
prec_prim_file_path = './az_prim_20_prec/az_prim_20_prec.shp'

block_gdf = import_block_file(block_cols, block_file_path)
precinct_gdf = import_prec_files(prec_gen_file_path, prec_prim_file_path)

File imports initiated
block shape:  (155444, 14)
precinct shape post merge:  (1489, 411)


In [9]:
#Create new pop variable for VAP minus prison population to approximate voting population
block_gdf['P0040001-P0050003'] = block_gdf['P0040001'] - block_gdf['P0050003']
print(len(block_gdf[block_gdf['P0050003']!=0]))
print(len(block_gdf[block_gdf['P0040001']!=block_gdf['P0040001-P0050003']]))

81
81


In [10]:
import warnings
warnings.filterwarnings('ignore')


def pre_maup_check(block_gdf, precinct_gdf):
    prec_geom = precinct_gdf.geometry
    valid_rows = precinct_gdf[~(prec_geom.isna() | prec_geom.is_empty)]
    print('valid precinct rows: ', valid_rows.shape)

    block_geom = block_gdf.geometry
    block_valid_rows = block_gdf[~(block_geom.isna() | block_geom.is_empty)]
    print('valid block rows: ', block_valid_rows.shape)


#Check imports
pre_maup_check(block_gdf, precinct_gdf)

valid precinct rows:  (1489, 411)
valid block rows:  (155444, 15)


In [11]:
def assign_votes(variables, election_columns, precinct_gdf, block_gdf):
    precinct_gdf[variables] = block_gdf[variables].groupby(block_gdf["maup_assignment"]).sum()
    print(variables, ' added to precinct_gdf based on maup assignment')
    bl_to_prec_weights = block_gdf[variables] / block_gdf["maup_assignment"].map(precinct_gdf[variables])
    block_votes = block_gdf
    block_votes[election_columns] = maup.prorate(
        block_gdf["maup_assignment"], precinct_gdf[election_columns], bl_to_prec_weights
    )
    return block_votes


def maup_assign_labels(block_gdf, precinct_gdf):
    block_gdf = block_gdf.to_crs(precinct_gdf.crs)
    print('block_gdf shape: ', block_gdf.shape, '\n precinct_gdf shape: ', precinct_gdf.shape)

    block_gdf["maup_assignment"] = maup.assign(
        fix_buffer(block_gdf), fix_buffer(precinct_gdf)
    )
    #Assign precinct IDs to block file using assign series
    block_gdf["PRECINCTID"] = block_gdf["maup_assignment"].map(
        lambda idx: str(precinct_gdf.loc[idx]["UNIQUE_ID"])
    )

    block_gdf["CON_DIST"] = block_gdf["maup_assignment"].map(
        lambda idx: str(precinct_gdf.loc[idx]["CON_DIST_x"])
    )

    block_gdf["SLDL_DIST"] = block_gdf["maup_assignment"].map(
        lambda idx: str(precinct_gdf.loc[idx]["SLDL_DIST_x"])
    )

    block_gdf["SLDU_DIST"] = block_gdf["maup_assignment"].map(
        lambda idx: str(precinct_gdf.loc[idx]["SLDU_DIST_x"])
    )
    print('PRECINCTIDs block file: ', block_gdf.PRECINCTID.nunique())

    return block_gdf


def fix_buffer(gdf):
    """
    return (GeoDataFrame) with the 'bufer(0) trick' applied
    :gdf: (GeoDataFrame) object
    Can be useful when trying to mitigate 'self-intersection' issues
    """
    buffered = gdf.buffer(0)
    gdf.drop(columns=["geometry"])
    # gdf['geometry'] = gdf.apply(lambda x: x.geometry.buffer(0), axis=1)
    gdf["geometry"] = buffered
    return gdf

In [12]:
#Assign identifiers
block_gdf = maup_assign_labels(block_gdf, precinct_gdf)

block_gdf shape:  (155444, 15) 
 precinct_gdf shape:  (1489, 411)


100%|██████████| 1489/1489 [00:20<00:00, 73.73it/s] 


PRECINCTIDs block file:  1489


In [13]:
#Assign votes
variables = 'P0040001-P0050003'
election_columns = list(fields_dict_1.keys()) + list(fields_dict_2.keys())
block_votes = assign_votes(variables, election_columns, precinct_gdf, block_gdf)

P0040001-P0050003  added to precinct_gdf based on maup assignment


In [22]:
block_gdf = block_votes
block_gdf["SLDL_DIST"] = block_gdf["SLDL_DIST"].str.zfill(3)
block_gdf["SLDU_DIST"] = block_gdf["SLDU_DIST"].str.zfill(3)

### Check district assignments

In [33]:
def district_mismatch_report(block_gdf, maup_dist_col, census_dist_col, state, district_type):
    b_mismatch = block_gdf[block_gdf[maup_dist_col]!=block_gdf[census_dist_col]]
    print('\n',district_type, "Block Impact:")
    print(b_mismatch.shape[0], " of the Census Blocks in ", state, " have mismatched ", district_type, " districts.")
    print("There are ", block_gdf.shape[0], " total Census Blocks in ", state, ". This means that ", round(100*b_mismatch.shape[0]/block_gdf.shape[0], 3), 
          "% of the blocks have mismatched ", district_type, " districts.")
    print("Of the ", b_mismatch.shape[0], ", ", b_mismatch[b_mismatch['P0040001']!=0].shape[0], " have a population greater than zero assigned and ", 
          b_mismatch[b_mismatch['P0040001']==0].shape[1], " are assigned 0 population by the Census, and therefore were not allocated any votes.")
    print("This means that only ", round(100*b_mismatch[b_mismatch['P0040001']!=0].shape[0]/block_gdf.shape[0], 3), "% of the blocks were actually impacted.")
    print("\n",district_type, "Population Impact:")
    print("The total Voting Age Population within the ", b_mismatch.shape[0], " blocks is ", b_mismatch['P0040001'].sum())
    print("The total prison population within the ", b_mismatch.shape[0], " blocks is ", b_mismatch['P0050003'].sum())
    print(state, "'s total Voting Age Population is ", block_gdf['P0040001'].sum())
    print("Therefore, ", round(100*b_mismatch['P0040001'].sum()/block_gdf['P0040001'].sum(), 3), 
          "% of the reported population is actually impacted by the district assignment discrepancy.")

In [34]:
district_mismatch_report(block_gdf, "CON_DIST", "CD116", "Arizona", "Congressional")
district_mismatch_report(block_gdf, "SLDL_DIST", "SLDL18", "Arizona", "SLDL")
district_mismatch_report(block_gdf, "SLDU_DIST", "SLDU18", "Arizona", "SLDU")


 Congressional Block Impact:
233  of the Census Blocks in  Arizona  have mismatched  Congressional  districts.
There are  155444  total Census Blocks in  Arizona . This means that  0.15 % of the blocks have mismatched  Congressional  districts.
Of the  233 ,  28  have a population greater than zero assigned and  412  are assigned 0 population by the Census, and therefore were not allocated any votes.
This means that only  0.018 % of the blocks were actually impacted.

 Congressional Population Impact:
The total Voting Age Population within the  233  blocks is  374
The total prison population within the  233  blocks is  0
Arizona 's total Voting Age Population is  5541976
Therefore,  0.007 % of the reported population is actually impacted by the district assignment discrepancy.

 SLDL Block Impact:
257  of the Census Blocks in  Arizona  have mismatched  SLDL  districts.
There are  155444  total Census Blocks in  Arizona . This means that  0.165 % of the blocks have mismatched  SLDL  di

### Export gdf

In [18]:
def format_gdf(block_votes, election_columns):
    block_votes['VAP_MOD'] = block_votes['P0040001-P0050003']
    ordered_cols = ['GEOID20', 'NAME20', 'STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'BLOCKCE20', 'PRECINCTID',
                    'CON_DIST', 'SLDL_DIST', 'SLDU_DIST', 'VAP_MOD'] + \
                   election_columns + \
                   ['ALAND20', 'AWATER20', 'geometry']
    block_votes_export = block_votes[ordered_cols]
    return block_votes_export


def create_shp(gdf, shp_name):
    os.mkdir('./' + shp_name)
    gdf.to_file('./' + shp_name + '/' + shp_name + '.shp')
    print(shp_name, 'shapefile created.')

In [19]:
block_votes_export = format_gdf(block_votes, election_columns).round(2)

In [20]:
block_votes_export.head()

,GEOID20,NAME20,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,PRECINCTID,CON_DIST,SLDL_DIST,SLDU_DIST,VAP_MOD,P20USSDKEL,P20USSRMCC,P20USSRMCS,PCON01DOHA,PCON01DPUT,PCON01RREI,PCON01RSHE,PCON02DKIR,PCON02DQUI,PCON02RMAR,PCON02RMOR,PCON02RRUD,PCON03DGRI,PCON03RWOO,PCON04DDIS,PCON04DSTA,PCON04RGOS,PCON04RWAR,PCON05DGRE,PCON05DIRE,PCON05DRAM,PCON05RBIG,PCON06DGEN,PCON06DMAL,PCON06DRIM,PCON06DTIP,PCON06RSCH,PCON07DGAL,PCON07RBAR,PCON08DMU1,PCON08DMU2,PCON08DOLS,PCON08RLES,PCON09DSTA,PCON09RGIL,PCON09RHUA,PCON09RTUT,P20COCDMUN,P20COCDSTA,P20COCDTOV,P20COCRMAR,P20COCRSLO,PSU01RFAN,PSU02DGAB,PSU02RWOR,PSU03DGON,PSU04DOTO,PSU04RANG,PSU05RBOR,PSU06DFRE,PSU06RALL,PSU06RROG,PSU07DPES,PSU08DMCG,PSU08RSHO,PSU09DSTE,PSU10DENG,PSU10RWAD,PSU11DMEN,PSU11DPAT,PSU11RLEA,PSU12DROB,PSU12RPET,PSU13RKER,PSU14DKAR,PSU14RGOW,PSU15RBAR,PSU15RCAR,PSU16RTOW,PSU17DKUR,PSU17RMES,PSU18DBOW,PSU18RSHA,PSU19DCHA,PSU20DERV,PSU20RBOY,PSU21RGRA,PSU22DTYR,PSU22RDIC,PSU22RLIV,PSU22RNGU,PSU23DBLA,PSU23RKOL,PSU23RUGE,PSU24DALS,PSU24DSTA,PSU25DWEI,PSU25RPAC,PSU26DGRA,PSU26DMEN,PSU26RCHI,PSU27DRIO,PSU27RSHR,PSU28DMAR,PSU28RBRO,PSU29DQUE,PSU29RWIL,PSU30DNAV,PSL01DSTA,PSL01RBLI,PSL01RBUR,PSL01RCOC,PSL01RNGU,PSL01RSEN,PSL02DDAL,PSL02DHER,PSL02DPAR,PSL02DPEA,PSL02RMCE,PSL03DCAN,PSL03DHER,PSL03DSOT,PSL04DFER,PSL04DPET,PSL04RJOH,PSL05RBIA,PSL05RCOB,PSL06DEVA,PSL06RBAR,PSL06RBLA,PSL07DTEL,PSL07DTSO,PSL07RPAR,PSL07RPEE,PSL08DGIR,PSL08RCAR,PSL08RCOO,PSL08RPRA,PSL09DFRI,PSL09DPOW,PSL09RLYO,PSL10DDEG,PSL10DSTA,PSL10RGUM,PSL10RHIC,PSL11DPER,PSL11RFIN,PSL11RROB,PSL12RGRA,PSL12RHOF,PSL13DSAN,PSL13RDUN,PSL13RMON,PSL13ROSB,PSL14DBEA,PSL14DMAE,PSL14RGRI,PSL14RNUT,PSL15DDYB,PSL15RHAM,PSL15RKAI,PSL15RWIL,PSL16RFIL,PSL16RGOD,PSL16RMOR,PSL16RPAR,PSL17DPAW,PSL17RHAR,PSL17RWEN,PSL18DEPS,PSL18DJER,PSL18RROB,PSL19DESP,PSL19DSIE,PSL19DSUN,PSL20DSCH,PSL20RBOL,PSL20RKER,PSL21DKNE,PSL21RMIL,PSL21RPAY,PSL21RPIN,PSL22DGAR,PSL22DHON,PSL22RCAR,PSL22RTOM,PSL23DKUR,PSL23RCHA,PSL23RKAV,PSL23RLAW,PSL24DLON,PSL24DSHA,PSL25DHUG,PSL25RBOW,PSL25RPEA,PSL25RUDA,PSL26DHER,PSL26DMOR,PSL26DNEZ,PSL26DSAL,PSL26RLOU,PSL26RSIF,PSL27DBOL,PSL27DMIR,PSL27DROD,PSL27RPEÑ,PSL28DBUT,PSL28DLIE,PSL28RBOW,PSL28RJAC,PSL29DAND,PSL29DCAS,PSL29DCHA,PSL29RBRA,PSL29RFOK,PSL29RMCM,PSL30DMEZ,PSL30DTER,G20PREDBID,G20PRELJOR,G20PRERTRU,G20USSDKEL,G20USSRMCS,GCON01DOHA,GCON01RSHE,GCON02DKIR,GCON02RMAR,GCON03DGRI,GCON03RWOO,GCON04DDIS,GCON04RGOS,GCON05DGRE,GCON05RBIG,GCON06DTIP,GCON06RSCH,GCON07DGAL,GCON07RBAR,GCON08DMUS,GCON08RLES,GCON09DSTA,GCON09RGIL,G20SSCNBRU,G20SSCNGOU,G20SSCNLOP,G20SSCYBRU,G20SSCYGOU,G20SSCYLOP,G20COCDMUN,G20COCDSTA,G20COCDTOV,G20COCRMAR,G20COCROCO,G20COCRSLO,G20PRO207N,G20PRO207Y,G20PRO208N,G20PRO208Y,GSU01DCAR,GSU01RFAN,GSU02DGAB,GSU02RWOR,GSU03DGON,GSU04DOTO,GSU04RANG,GSU05RBOR,GSU06DFRE,GSU06RROG,GSU07DPES,GSU08DMCG,GSU08RSHO,GSU09DSTE,GSU10DENG,GSU10RWAD,GSU11DMEN,GSU11RLEA,GSU12DROB,GSU12RPET,GSU13RKER,GSU14DKAR,GSU14RGOW,GSU15RBAR,GSU16RTOW,GSU17DKUR,GSU17RMES,GSU18DBOW,GSU18RSHA,GSU19DCHA,GSU20DERV,GSU20RBOY,GSU21RGRA,GSU22DTYR,GSU22RLIV,GSU23DBLA,GSU23RUGE,GSU24DALS,GSU24RMIC,GSU25DWEI,GSU25RPAC,GSU26DMEN,GSU26RCHI,GSU27DRIO,GSU27RSHR,GSU28DMAR,GSU28RBRO,GSU29DQUE,GSU29RWIL,GSU30DNAV,GSL01DSTA,GSL01RBUR,GSL01RNGU,GSL02DDAL,GSL02DHER,GSL02RMCE,GSL03DCAN,GSL03DHER,GSL04DFER,GSL04DPET,GSL04RJOH,GSL05RBIA,GSL05RCOB,GSL06DEVA,GSL06IBAB,GSL06RBAR,GSL06RBLA,GSL07DTEL,GSL07DTSO,GSL07RPAR,GSL07RPEE,GSL08DGIR,GSL08RCOO,GSL08RPRA,GSL09DFRI,GSL09DPOW,GSL09RLYO,GSL10DDEG,GSL10DSTA,GSL10RGUM,GSL10RHIC,GSL11DPER,GSL11RFIN,GSL11RROB,GSL12RGRA,GSL12RHOF,GSL13DSAN,GSL13RDUN,GSL13ROSB,GSL14DBEA,GSL14DMAE,GSL14RGRI,GSL14RNUT,GSL15DDYB,GSL15RKAI,GSL15RWIL,GSL16DHUN,GSL16RFIL,GSL16RPAR,GSL17DPAW,GSL17RHAR,GSL17RWEN,GSL18DEPS,GSL18DJER,GSL18RHAW,GSL18RROB,GSL19DESP,GSL19DSIE,GSL20DSCH,GSL20RBOL,GSL20RKER,GSL21DKNE,GSL21RPAY,GSL21RPIN,GSL22DGAR,GSL22DHON,GSL22RCAR,GSL22RTOM,GSL23DKUR,GSL23RCHA,GSL23RKAV,GSL24DLON,GSL24DSHA,GSL24RALG,GSL24RCUS,GSL25DHUG,GSL25RBOW,GSL25RUDA,GSL26DHER,GSL26DSAL,GSL26RLOU,GSL26RSIF,GSL27DBOL,GSL27DROD,GSL2

In [21]:
# Export block file
shp_name = 'az_20_block'
create_shp(block_votes_export, shp_name)

az_20_block shapefile created.


### Validation

In [35]:
#Function to check column/race totals
def column_total_check(election_columns, block_gdf, precinct_gdf):
    for val in election_columns:
        vote_dif = block_gdf[val].sum()-precinct_gdf[val].sum()
        if (vote_dif == 0):
            print(val+": EQUAL", ' - total: ', str(block_gdf[val].sum()))
        else:
            print(val+": DIFFERENCE OF " + str(vote_dif)+ " VOTES", ' - block total: ', str(block_gdf[val].sum()), ', precinct total: ', str(precinct_gdf[val].sum()))     
            
#Function to check county totals
def county_total_check(election_columns, block_county_fp_col, prec_county_fp_col, block_gdf, precinct_gdf):
    print("Counties with differences printed below:")
    diff_counties=[]
    for i in election_columns:
        diff = block_gdf.groupby([block_county_fp_col]).sum()[i]-precinct_gdf.groupby([prec_county_fp_col]).sum()[i]
        for val in diff[diff != 0].index.values.tolist():
            if val not in diff_counties:
                diff_counties.append(val)
        if len(diff[diff != 0]!=0):
            print(i)
            print(diff[diff != 0].to_string(header=False))
    print("")
    print("All other races in all counties are equal")
    
#Reaggregate results and check totals
def reagg_and_check(block_votes, bl_pr_assign, precinct_gdf):
    agg = block_votes.groupby(bl_pr_assign).sum()
    column_total_check(election_columns, agg, precinct_gdf)

In [36]:
#Check column/race totals
column_total_check(election_columns, block_votes, precinct_gdf)

P20USSDKEL: EQUAL  - total:  665927.0
P20USSRMCC: DIFFERENCE OF -2.9103830456733704e-11 VOTES  - block total:  181666.99999999997 , precinct total:  181667.0
P20USSRMCS: DIFFERENCE OF -1.1641532182693481e-10 VOTES  - block total:  551400.9999999999 , precinct total:  551401.0
PCON01DOHA: EQUAL  - total:  47083.0
PCON01DPUT: EQUAL  - total:  33248.0
PCON01RREI: EQUAL  - total:  33418.0
PCON01RSHE: DIFFERENCE OF -7.275957614183426e-12 VOTES  - block total:  40309.99999999999 , precinct total:  40310.0
PCON02DKIR: DIFFERENCE OF -1.4551915228366852e-11 VOTES  - block total:  77516.99999999999 , precinct total:  77517.0
PCON02DQUI: DIFFERENCE OF 3.637978807091713e-12 VOTES  - block total:  24035.000000000004 , precinct total:  24035.0
PCON02RMAR: DIFFERENCE OF -3.637978807091713e-12 VOTES  - block total:  31729.999999999996 , precinct total:  31730.0
PCON02RMOR: DIFFERENCE OF 3.637978807091713e-12 VOTES  - block total:  17802.000000000004 , precinct total:  17802.0
PCON02RRUD: DIFFERENCE OF

In [37]:
#Check county totals
block_county_fp_col = "COUNTYFP20"
prec_county_fp_col = "COUNTYFP_x"
county_total_check(election_columns, block_county_fp_col, prec_county_fp_col, block_votes, precinct_gdf)

Counties with differences printed below:
P20USSDKEL
001   -2.000888e-11
003    3.637979e-12
005    2.182787e-11
007   -5.456968e-12
009   -2.273737e-12
011    1.023182e-12
012   -1.591616e-12
013   -3.492460e-10
015    7.457857e-11
017    8.913048e-11
019   -2.328306e-10
021    7.275958e-11
023    1.273293e-11
025    3.274181e-11
027    3.092282e-11
P20USSRMCC
001   -3.410605e-12
003   -3.637979e-12
005    1.818989e-12
007    4.547474e-12
009   -6.821210e-13
011   -1.421085e-13
012    4.547474e-13
013   -1.222361e-09
015    9.094947e-13
017   -2.000888e-11
019   -8.731149e-11
021   -2.910383e-11
023   -3.410605e-13
025    4.911271e-11
027   -4.092726e-12
P20USSRMCS
001   -1.136868e-11
003   -2.728484e-11
005    1.273293e-11
007   -3.637979e-12
009    4.547474e-13
011   -1.136868e-13
012   -1.136868e-12
013    3.143214e-09
015    2.182787e-11
017    5.820766e-11
019    1.891749e-10
021    1.164153e-10
023    2.955858e-12
025   -3.637979e-11
027    6.366463e-11
PCON01DOHA
001   -7.275958

In [ ]:
#check reagg
reagg_and_check(block_votes, block_votes['maup_assignment'], precinct_gdf)